In [1]:
import pandas as pd
import pymysql
from pymysql.err import Error

In [3]:
# create database connection

def create_server_connection(host_name, user_name, user_password):
    """
    Establishes a connection to the MySQL server using the provided credentials.

    Parameters:
    host_name (str): The hostname or IP address of the MySQL server.
    user_name (str): The MySQL username used to authenticate.
    user_password (str): The password associated with the MySQL user.

    Returns:
    connection (pymysql.connect object): A MySQL connection object if the connection is successful.
    None: If the connection fails, an error message is printed, and None is returned.
    """
    connection = None  #Initialize the connection object as None before attempting the connection
    try:
        # Attempt to establish a connection to the MySQL server with the provided credentials.
        connection = pymysql.connect(
            host = host_name,                    #Hostname or IP of the server
            user = user_name,                    #MySQL username
            passwd = user_password               #Password for the MySQL user
        )
        print("Connection Sucessfull")          #Log message indicating that the connection was successful
    
     # Catch any exceptions (Errors) that occur during the connection attempt
    except Error as err:
        print(f"Error: '{err}'")
        
    return connection  #Return the connection object (None if the connection failed)


passwd = 'NA'

connection = create_server_connection("localhost","root",passwd)
        

Connection Sucessfull


In [4]:
#create database

def create_database(connection, db_name):
    """
    Function to create a new database if it does not already exist.
    
    Parameters:
    - connection: Active connection object to the MySQL server.
    - db_name: Name of the database to create.
    """
    # Create a cursor object to interact with the MySQL server
    cursor = connection.cursor()
    try:
        # Check if the database already exists
        cursor.execute(f"SHOW DATABASES LIKE '{db_name}'")
        result = cursor.fetchone()
        
        # Only create the database if it does not exist
        if not result:
            cursor.execute(f"CREATE DATABASE {db_name}")
            print(f"Database '{db_name}' created successfully")
        else:
            print(f"Database '{db_name}' already exists")
    except Error as err:
        print(f"Error: '{err}'")

# Define the database name
db_name = "mysql_python"

# Call the function to check and create the database
create_database(connection, db_name)


Database 'mysql_python' already exists


In [5]:
# connect to database
# Function to connect to a specified MySQL database
def connect_db_connection(host_name, user_name, user_password, database_name):
    """
    Establishes a connection to a MySQL database and returns the connection object.
    
    Parameters:
    - host_name (str): Hostname or IP address of the MySQL server.
    - user_name (str): Username to authenticate with the MySQL server.
    - user_password (str): Password for the specified username.
    - database_name (str): Name of the database to connect to.

    Returns:
    - connection (pymysql.connections.Connection): Connection object if successful, None otherwise.
    """
    connection = None
    try:
        # Establish connection to the MySQL server
        connection = pymysql.connect(
            host=host_name,              # Hostname or IP of the server
            user=user_name,              # MySQL username
            passwd=user_password,        # Password for the MySQL user
            db=database_name             # Name of the specific database to connect to
        )
        print("Connection successful")   # Log message indicating that the connection was successful

    except Error as err:
        # Catch and print any exception (Error) that occurs during the connection attempt
        print(f"Error: '{err}'")
        
    return connection  # Return the connection object (None if the connection failed)

# Attempt to connect to the database
connection = connect_db_connection("localhost", "root", "NA", "mysql_python")


Connection successful


In [6]:
# Function to execute SQL queries on the connected MySQL database
def execute_query(connection, query):
    """
    Executes a given SQL query using the provided database connection.
    
    Parameters:
    - connection (pymysql.connections.Connection): Active connection to the MySQL database.
    - query (str): SQL query to be executed.
    
    Returns:
    - None
    """
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    try:
        # Execute the SQL query
        cursor.execute(query)
        
        # Commit the transaction to make sure the changes are saved
        connection.commit()
        print("Query executed successfully")
        
    except pymysql.MySQLError as err:
        # Catch and display any error that occurs during query execution
        print(f"Error: '{err}'")


In [7]:
# Define the SQL query to create the "orders" table
create_orders_table = """
CREATE TABLE IF NOT EXISTS orders (
    order_id INT PRIMARY KEY,
    customer_name VARCHAR(30) NOT NULL,
    product_name VARCHAR(20) NOT NULL,
    date_ordered DATE,
    quantity INT,
    unit_price FLOAT,
    phone_number VARCHAR(20)
);
"""

# Define your database credentials
host_name = "localhost"
user_name = "root"
password = "NA"  # Replace 'your_password' with the actual password
database_name = "mysql_python"  # Ensure this database exists

# Connect to the database
connection = connect_db_connection(host_name, user_name, password, database_name)

# Execute the query to create the table
execute_query(connection, create_orders_table)

# Close the database connection after query execution
connection.close()


Connection successful
Query executed successfully


In [8]:
# Define the SQL query to insert data into the "orders" table
data_orders = """
INSERT INTO orders VALUES
(101, 'Steve', 'Laptop', '2018-06-12', 2, 800, '6293730802'),
(102, 'Jos', 'Books', '2019-02-10', 10, 12, '8367489124'),
(103, 'Stacy', 'Trousers', '2019-12-25', 5, 50, '8976123645'),
(104, 'Nancy', 'T-Shirts', '2018-07-14', 7, 30, '7368145099'),
(105, 'Maria', 'Headphones', '2019-05-30', 6, 48, '8865316698'),
(106, 'Danny', 'Smart TV', '2018-08-20', 10, 300, '7720130449');
"""

# Connect to the database
connection = connect_db_connection("localhost", "root", "NA", "mysql_python")

# Execute the query to insert data
execute_query(connection, data_orders)

# Close the database connection after query execution
connection.close()


Connection successful
Error: '(1062, "Duplicate entry '101' for key 'orders.PRIMARY'")'


In [9]:
# Function to read data from a MySQL database
def read_query(connection, query):
    """
    Executes a SELECT query and retrieves data from the database.
    
    Parameters:
    - connection (pymysql.connections.Connection): Active connection to the MySQL database.
    - query (str): SQL SELECT query to be executed.
    
    Returns:
    - result (list of tuples): Fetched records from the query, or None if an error occurs.
    """
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    result = None
    try:
        # Execute the SELECT query
        cursor.execute(query)
        
        # Fetch all results from the executed query
        result = cursor.fetchall()
        return result  # Return the fetched results as a list of tuples
        
    except pymysql.MySQLError as err:
        # Catch and display any error that occurs during query execution
        print(f"Error: '{err}'")


In [10]:
# Define the SQL query to select all data from the orders table
q1 = """
SELECT * FROM orders;
"""

# Connect to the database
connection = connect_db_connection("localhost", "root", "NA", "mysql_python")

# Execute the query and retrieve results
results = read_query(connection, q1)

# Print each record in the results
for result in results:
    print(result)

# Close the database connection after query execution
connection.close()


Connection successful
(101, 'Steve', 'Laptop', datetime.date(2018, 6, 12), 2, 800.0, '6293730802')
(102, 'Jos', 'Books', datetime.date(2019, 2, 10), 10, 12.0, '8367489124')
(103, 'Stacy', 'Trousers', datetime.date(2019, 12, 25), 5, 50.0, '8976123645')
(104, 'Nancy', 'T-Shirts', datetime.date(2018, 7, 14), 7, 30.0, '7368145099')
(105, 'Maria', 'Headphones', datetime.date(2019, 5, 30), 6, 48.0, '8865316698')
(106, 'Danny', 'Smart TV', datetime.date(2018, 8, 20), 10, 300.0, '7720130449')


In [12]:
from_db = []  # Initialize an empty list to store rows from the database query

# Iterate through each result tuple from the SQL query results
for result in results:
    result = list(result)  # Convert each tuple to a list to ensure compatibility with DataFrame
    from_db.append(result)  # Append the list to the 'from_db' list

# Define column names for the DataFrame
columns = ["order_id", "customer_name", "product_name", "date_ordered", "quantity", "unit_price", "phone_number"]

# Create a DataFrame from the list of rows and the specified column names
df = pd.DataFrame(from_db, columns=columns)

# Display the resulting DataFrame
display(df)


,order_id,customer_name,product_name,date_ordered,quantity,unit_price,phone_number
0,101,Steve,Laptop,2018-06-12,2,800.0,6293730802
1,102,Jos,Books,2019-02-10,10,12.0,8367489124
2,103,Stacy,Trousers,2019-12-25,5,50.0,8976123645
3,104,Nancy,T-Shirts,2018-07-14,7,30.0,7368145099
4,105,Maria,Headphones,2019-05-30,6,48.0,8865316698
5,106,Danny,Smart TV,2018-08-20,10,300.0,7720130449
